In [1]:
# import widen_notebook
from mysetup import NotebookFinder
import sys

sys.meta_path.append(NotebookFinder())

from setup_transform import *
     
%matplotlib inline
cl_weight = sklearn.utils.compute_class_weight(
    class_weight="balanced", classes=[0, 1, 2], y=y
)
CLASS_WEIGHTS = {i: cl_weight[i] for i in range(3)}
def_cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5, random_state=42)
lgr_params = dict(
    class_weight=CLASS_WEIGHTS,
    fit_intercept=False,
    multi_class="ovr",
    max_iter=2000000,
    random_state=42,
    n_jobs=24,
    #     penalty="elasticnet",
    cv=def_cv,
    scoring="f1_macro",
    solver="lbfgs",
    Cs=100,
    #     l1_ratios=np.linspace(0, 1, endpoint=False, num=100),
)
from sklearnex import unpatch_sklearn

import copy
import warnings

warnings.filterwarnings("ignore")
discrete = nominal + discrete_ordinal + discrete_binary
X_master = pd.concat(
    [raw_data.loc[:, raw_data_eval.columns], raw_data_eval], ignore_index=True, axis=0
)
class_priors = (raw_data.target.value_counts() / 3796).to_numpy()
raw_data[discrete] = raw_data[discrete].astype(np.uint32)
X_master[discrete] = X_master[discrete].astype(np.uint32)
numeric = list(
    np.setdiff1d(raw_data_eval.columns, discrete_binary + discrete_ordinal + nominal+['target'])
)
for c in raw_data_eval.columns:
    if c not in numeric:
        raw_data[c] = raw_data[c].astype(np.uint32)
        raw_data_eval[c] = raw_data_eval[c].astype(np.uint32)
raw_data['target'] = raw_data['target'].astype(np.uint8)


importing Jupyter notebook from setup_transform.ipynb


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [13]:
from sklearn.utils.validation import has_fit_parameter

# has_fit_parameter(clf, "warm_start")
clf_list = sklearn.utils.all_estimators(type_filter="classifier")
props = {"warm_start": [], "partial_fit": []}
errors = []
for name, clf in clf_list:
    try:
        if "warm_start" in clf().get_params():
            props["warm_start"].append(clf)
        try:
            clf().partial_fit(
                raw_data[raw_data_eval.columns].abs(),
                raw_data.target,
                classes=[0, 1, 2],
            )
            props["partial_fit"].append(clf)
        except AttributeError:
            errors.append(name)

    except TypeError:
        errors.append(name)

In [16]:
props

{'warm_start': [sklearn.ensemble._bagging.BaggingClassifier,
  sklearn.ensemble._forest.ExtraTreesClassifier,
  sklearn.gaussian_process._gpc.GaussianProcessClassifier,
  sklearn.ensemble._gb.GradientBoostingClassifier,
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier,
  daal4py.sklearn.linear_model.logistic_path.LogisticRegression,
  sklearn.neural_network._multilayer_perceptron.MLPClassifier,
  sklearn.linear_model._passive_aggressive.PassiveAggressiveClassifier,
  sklearn.linear_model._perceptron.Perceptron,
  daal4py.sklearn.ensemble._forest.RandomForestClassifier,
  sklearn.linear_model._stochastic_gradient.SGDClassifier],
 'partial_fit': [sklearn.naive_bayes.BernoulliNB,
  sklearn.naive_bayes.CategoricalNB,
  sklearn.naive_bayes.ComplementNB,
  sklearn.naive_bayes.GaussianNB,
  sklearn.neural_network._multilayer_perceptron.MLPClassifier,
  sklearn.naive_bayes.MultinomialNB,
  sklearn.linear_model._passive_aggressive.PassiveAggressiveClas

In [55]:
def get_logit_ordinal(X):
    probit = {c: {} for c in discrete_ordinal}

    for c in discrete_ordinal:
        vc = X_master[c].value_counts()
        N = X_master.shape[0]
        vc = vc / N
        probit_value = {val: 0 for val in X_master[c].unique()}
        for val in X_master[c].unique():
            probit_value[val] = np.log1p(
                vc[vc.index <= val].sum() / (vc[vc.index > val].sum() + 1e-09)
            )
        probit[c] = probit_value

    def transform_ordinal_to_logit(row):
        for c in discrete_ordinal:
            row[c] = probit[c][row[c]]
        return row

    probit_x = pd.DataFrame(X, columns=discrete_ordinal)
    probit_x.apply(transform_ordinal_to_logit, axis=1)
    return probit_x[discrete_ordinal].to_numpy()


base_n = BaseNEncoder(cols=nominal,handle_unknown=0)
numeric = list(
    np.setdiff1d(raw_data_eval.columns, discrete_binary + discrete_ordinal + nominal)
)
tree_ready = make_column_transformer(
    (base_n, nominal),
    (
        Normalizer(),
        numeric,
    ),
    ("passthrough", discrete_binary + discrete_ordinal),
    remainder="passthrough",
)


linear_ready = make_column_transformer(
    #     (OneHotEncoder(sparse=False),nominal),
    (BaseNEncoder(cols=nominal,handle_unknown=0), nominal),
    (
        FunctionTransformer(get_logit_ordinal, feature_names_out="one-to-one"),
        discrete_ordinal,
    ),
    (StandardScaler(), numeric),
    remainder="passthrough",
)

ohe = OneHotEncoder(sparse=False,)
ohe.fit(X_master[nominal])
nom_categories = ohe.categories_
bayesian_prep = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(sparse=False,categories=nom_categories), nominal),
        ("passthrough", discrete_binary + discrete_ordinal),
        remainder="drop",
        sparse_threshold=0,
    ),
    FunctionTransformer(np.abs)
    
)
# bayesian_prep.fit_transform(raw_data)

In [ ]:
X = raw_data[raw_data_eval.columns]
y = raw_data.target
X_eval = raw_data_eval
X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=0.3)
c_msk = np.array([True] * 44 + [False] * 15 + [True] * 22)
min_c = pd.DataFrame(bayesian_prep.fit_transform(X)).max().astype(np.uint32).to_list()
min_c = [mc+1 for mc in min_c]
c_msk = c_msk.reshape(
    81,
)
svc = SVC(
    probability=True,
    class_weight=CLASS_WEIGHTS,
    random_state=42,
    tol=1e-06,
    kernel="rbf",
    gamma="scale",
    break_ties=True,
)
sgd = SGDClassifier(
    warm_start=True,
    average=True,
    random_state=42,
    fit_intercept=False,
    n_iter_no_change=50,
    loss="squared_hinge",
    max_iter=10000,class_weight=CLASS_WEIGHTS
)
hgb = HistGradientBoostingClassifier(
    warm_start=True,
    categorical_features=c_msk,
    random_state=42,
    scoring="f1_macro",
    max_iter=200,
    early_stopping="auto",
    n_iter_no_change=50,

)
rfc = RandomForestClassifier(
    warm_start=True,
    min_impurity_decrease=0.00055,
    n_jobs=24,
    random_state=42,
    max_features=None,
    class_weight=CLASS_WEIGHTS
)
efc = ExtraTreesClassifier(
    warm_start=True,
    min_impurity_decrease=0.00055,
    n_jobs=24,
    random_state=42,
    max_features=None,
    class_weight=CLASS_WEIGHTS
)
y_evals = []
ALPHA = 1.0e-10
cnb = CategoricalNB(alpha=ALPHA, fit_prior=True, min_categories=min_c)
conb = ComplementNB(alpha=ALPHA, fit_prior=True, norm=False)
benb = BernoulliNB(binarize=False, alpha=ALPHA, fit_prior=True)
munb = MultinomialNB(alpha=ALPHA, fit_prior=True)


b_clfs = []
bayesian_model_data = bayesian_prep.fit_transform(X,y)
bayesian_model_eval = bayesian_prep.transform(X_eval)

t_clfs = []
tree_model_data = tree_ready.fit_transform(X,y)
tree_model_eval = tree_ready.transform(X_eval)

l_clfs = []
linear_model_data = linear_ready.fit_transform(X,y)
linear_model_eval = linear_ready.transform(X_eval)

for bayesian_learner in [conb, benb, munb]:
    clf_ = BaggingClassifier(
        base_estimator=bayesian_learner,
        n_estimators=10,
        random_state=42,
        n_jobs=24,
        warm_start=True,
        bootstrap=False
    )
    b_clfs.append((bayesian_learner.__class__.__name__, bayesian_learner))

for tree_learner in [hgb, rfc, efc]:
    clf_ = BaggingClassifier(
        base_estimator=tree_learner,
        n_estimators=10,
        random_state=42,
        n_jobs=24,
        bootstrap=False,
        warm_start=True,
    )
    t_clfs.append((tree_learner.__class__.__name__, tree_learner))

for linear_learner in [svc, sgd]:
    clf_ = BaggingClassifier(
        base_estimator=linear_learner,
        n_estimators=10,
        random_state=42,
        n_jobs=24,
        bootstrap=False,
        warm_start=True,
    )
    l_clfs.append((linear_learner.__class__.__name__, linear_learner))



VP = VotingClassifier(
    estimators=b_clfs + t_clfs+ l_clfs,
    voting="soft",
)

for clf_name,clf in b_clfs:
    X_train, X_test, y_train, y_test = gen_train_test(bayesian_model_data, y, test_size=0.3)
    print(clf_name)
    y_pred_base = clf.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, y_pred_base))
    print(":"*80)
    
for clf_name,clf in t_clfs:
    X_train, X_test, y_train, y_test = gen_train_test(tree_model_data, y, test_size=0.3)
    print(clf_name)
    y_pred_base = clf.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, y_pred_base))
    print(":"*80)
    
for clf_name,clf in l_clfs:
    X_train, X_test, y_train, y_test = gen_train_test(linear_model_data, y, test_size=0.3)
    print(clf_name)
    y_pred_base = clf.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, y_pred_base))
    print(":"*80)

In [93]:
sgd = SGDClassifier(
    warm_start=True,
    average=True,
    random_state=42,
    fit_intercept=False,
    n_iter_no_change=50,
    loss="squared_hinge",
    max_iter=10000,class_weight=CLASS_WEIGHTS
)
hgb = HistGradientBoostingClassifier(
    warm_start=True,
    categorical_features=c_msk,
    random_state=42,
    scoring="f1_macro",
    max_iter=200,
    early_stopping="auto",
    n_iter_no_change=50,

)
rfc = RandomForestClassifier(
    warm_start=True,
    min_impurity_decrease=0.00055,
    n_jobs=24,
    random_state=42,
    max_features=None,
    class_weight=CLASS_WEIGHTS
)
efc = ExtraTreesClassifier(
    warm_start=True,
    min_impurity_decrease=0.00055,
    n_jobs=24,
    random_state=42,
    max_features=None,
    class_weight=CLASS_WEIGHTS
)
svc_ = SVC(
    probability=True,
    class_weight=CLASS_WEIGHTS,
    random_state=42,
    tol=1e-06,
    kernel="rbf",
    gamma="scale",
    break_ties=True,
)

svc =BaggingClassifier( base_estimator=svc_,n_estimators=10,
        random_state=42,
        n_jobs=24,
        bootstrap=False,
        warm_start=True, )
cnb = CategoricalNB(alpha=ALPHA, fit_prior=True, min_categories=min_c)
conb = ComplementNB(alpha=ALPHA, fit_prior=True, norm=False)
benb = BernoulliNB(binarize=False, alpha=ALPHA, fit_prior=True)
munb = MultinomialNB(alpha=ALPHA, fit_prior=True)
def print_clf_perf(clf,X_test,y_test):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__)
    print(classification_report(y_test, y_pred))
    print(":"*80)

In [ ]:
with parallel_backend('threading',n_jobs=24):  
    X_train, X_test, y_train, y_test = gen_train_test(bayesian_model_data, y, test_size=0.3)
#     y_pred_base = clf.fit(X_train, y_train).predict(X_test)
    cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=10)
    for train_index, test_index in cv.split(X_train, y_train):
        X_train_, X_test_ = bayesian_model_data[train_index, :], bayesian_model_data[test_index, :]
        y_train_, y_test_ = y.loc[train_index], y.loc[test_index]
        #     plug.fit(X_test_, y_test_)
        cnb.partial_fit(X_train_, y_train_, classes=[0,1,2])
        cnb.partial_fit(X_test_, y_test_, classes=[0,1,2])
        conb.partial_fit(X_train_, y_train_, classes=[0,1,2])
        conb.partial_fit(X_test_, y_test_, classes=[0,1,2])
        benb.partial_fit(X_train_, y_train_, classes=[0,1,2])
        benb.partial_fit(X_test_, y_test_, classes=[0,1,2])
        munb.partial_fit(X_train_, y_train_, classes=[0,1,2])
        munb.partial_fit(X_test_, y_test_, classes=[0,1,2])
    print_clf_perf(cnb,X_test,y_test)
    print_clf_perf(conb,X_test,y_test)
    print_clf_perf(benb,X_test,y_test)
    print_clf_perf(munb,X_test,y_test)
    

#     for clf_name,clf in t_clfs:
    X_train, X_test, y_train, y_test = gen_train_test(tree_model_data, y, test_size=0.3)
    cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=10)
    for train_index, test_index in cv.split(X_train, y_train):
        X_train_, X_test_ = tree_model_data[train_index, :], tree_model_data[test_index, :]
        y_train_, y_test_ = y.loc[train_index], y.loc[test_index]
        #     plug.fit(X_test_, y_test_)
        hgb.fit(X_train_, y_train_)
        hgb.fit(X_test_, y_test_)
        rfc.fit(X_train_, y_train_)
        rfc.fit(X_test_, y_test_)
        efc.fit(X_train_, y_train_)
        efc.fit(X_test_, y_test_)
    print_clf_perf(hgb,X_test,y_test)
    print_clf_perf(rfc,X_test,y_test)
    print_clf_perf(efc,X_test,y_test)
    

#     for clf_name,clf in l_clfs:
    X_train, X_test, y_train, y_test = gen_train_test(linear_model_data, y, test_size=0.3)
    cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=10)
    for train_index, test_index in cv.split(X_train, y_train):
        X_train_, X_test_ = linear_model_data[train_index, :], linear_model_data[test_index, :]
        y_train_, y_test_ = y.loc[train_index], y.loc[test_index]
        #     plug.fit(X_test_, y_test_)
        sgd.fit(X_train_, y_train_)
        sgd.fit(X_test_, y_test_)
        svc.fit(X_train_, y_train_)
        svc.fit(X_test_, y_test_)
    print_clf_perf(sgd,X_test,y_test)
    print_clf_perf(svc,X_test,y_test)
[hgb,sgd,rfc,efc,benb,munb,conb,cnb,hgb] 

CategoricalNB
              precision    recall  f1-score   support

         0.0       0.77      0.67      0.72       373
         1.0       0.78      0.81      0.79       559
         2.0       0.49      0.55      0.51       207

    accuracy                           0.72      1139
   macro avg       0.68      0.68      0.68      1139
weighted avg       0.72      0.72      0.72      1139

::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
ComplementNB
              precision    recall  f1-score   support

         0.0       0.64      0.70      0.67       373
         1.0       0.67      0.84      0.74       559
         2.0       0.62      0.07      0.13       207

    accuracy                           0.65      1139
   macro avg       0.64      0.54      0.51      1139
weighted avg       0.65      0.65      0.61      1139

::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
BernoulliNB
              precision    recall  f1